In [1]:
import os
import numpy as np
import pandas as pd
from datetime import date

from sklearn.model_selection import KFold, train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, roc_auc_score, auc, roc_curve
from sklearn.preprocessing import MinMaxScaler

DATA_ROOT = "input/"

# 欄位說明 (與 column_description.csv 相同) 
### User_id：用戶 ID 
### Merchant_id：商家 ID 
### Coupon_id：優惠券 ID (null 代表無優惠券消費) 
### Discount_rate：優惠券折價：[0,1] 代表折扣率；x:y 代表滿 x 減 y 元 
### Distance：用戶經常活動地點離商家最近距離 (x * 500 公尺), 0 表示低於 500 公尺, 10 表示大於 5 公里。 
### Date_received：優惠券取得時間。 
### Date：購買商品時間 
            如果 Date is null & Coupon_id is not null, 則該紀錄為有優惠券但未使用; 
            若為 Date is not null & Coupon_id is null, 則為普通消費日期;
            若 Date is not null & Coupon_id is not null, 則表示優惠券消費日期)

In [2]:
dfoff = pd.read_csv(os.path.join(DATA_ROOT,'train_offline.csv'))
dftest = pd.read_csv(os.path.join(DATA_ROOT,'test_offline.csv'))
dftest = dftest[~dftest.Coupon_id.isna()]
dftest.reset_index(drop=True, inplace=True)
print(dfoff.shape)
print(dftest.shape)
dfoff.head(20)

(1160742, 7)
(306313, 6)


,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN
5,2223968,3381,9776.0,10:5,2.0,20160129.0,NaN
6,73611,2099,12034.0,100:10,NaN,20160207.0,NaN
7,163606,1569,5054.0,200:30,10.0,20160421.0,NaN
8,3273056,4833,7802.0,200:20,10.0,20160130.0,NaN
9,94107,3381,7610.0,200:20,2.0,20160412.0,NaN


In [3]:
dftest.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received
0,1439408,4663,11002.0,150:20,1.0,20160528.0
1,1439408,2632,8591.0,20:1,0.0,20160613.0
2,1439408,2632,8591.0,20:1,0.0,20160516.0
3,2029232,450,1532.0,30:5,0.0,20160530.0
4,2029232,6459,12737.0,20:1,0.0,20160519.0


In [4]:
## Creat target label 
"""
According to the definition, 
1) buy with coupon within (include) 15 days ==> 1
2) buy with coupon but out of 15 days ==> 0
3) buy without coupon ==> -1 (we don't care)
"""
def label(row):
    if np.isnan(row['Date_received']):
        return -1
    if not np.isnan(row['Date']):
        td = pd.to_datetime(row['Date'], format='%Y%m%d') -  pd.to_datetime(row['Date_received'], format='%Y%m%d')
        if td <= pd.Timedelta(15, 'D'):
            return 1
    return 0

dfoff["label"] = dfoff.apply(label, axis=1)
dfoff["label"].value_counts()

 0    710665
-1    413773
 1     36304
Name: label, dtype: int64

In [5]:
tmp = pd.to_datetime('20160519.0', format='%Y%m%d', errors='ignore')
time = [tmp.year , tmp.month, tmp.day]
time

[2016, 5, 19]

In [6]:
# Generate features - weekday acquired coupon
def getWeekday(row):
    if (np.isnan(row)) or (row==-1):
        return row
    else:
        return pd.to_datetime(row, format = "%Y%m%d").dayofweek+1 # add one to make it from 0~6 -> 1~7
    
            
dfoff['weekday'] = dfoff['Date_received'].apply(getWeekday)
dftest['weekday'] = dftest['Date_received'].apply(getWeekday)


# weekday_type (weekend = 1)
dfoff['weekday_type'] = dfoff['weekday'].astype('str').apply(lambda x : 1 if x in [6,7] else 0 ) # apply to trainset
dftest['weekday_type'] = dftest['weekday'].astype('str').apply(lambda x : 1 if x in [6,7] else 0 ) # apply to testset

In [7]:
weekdaycols = ['weekday_' + str(i) for i in range(1,8)]
print(weekdaycols)

tmpdf = pd.get_dummies(dfoff['weekday'].replace(-1, np.nan))
tmpdf.columns = weekdaycols
dfoff[weekdaycols] = tmpdf

tmpdf = pd.get_dummies(dftest['weekday'].replace(-1, np.nan))
tmpdf.columns = weekdaycols
dftest[weekdaycols] = tmpdf

['weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']


In [8]:
# Generate features - check if used Coupon ontime
def getUsedOntime(row):
    if (np.isnan(row)) or (row==-1):
        return 0
    else:
        time = pd.to_datetime(row, format = "%Y%m%d")
        if time.year != 2016 or time.month < 5:
            return 0
        elif time.month == 5 and time.day < 16:
            return 0
        else:
            return 1

dfoff['usedOntime'] = dfoff['Date_received'].apply(getUsedOntime)
dftest['usedOntime'] = dftest['Date_received'].apply(getUsedOntime)

In [9]:
# Generate features - 因為 test 沒有 Date 這項feature不成立（有Date＝有答案了）
 
#def getCouponUsage(row):
#    if np.isnan(row['Date']) | np.isnan(row['Coupon_id']): 
        ### 如果 Date is null & Coupon_id is not null, 則該紀錄為有優惠券但未使用;
        ### 若為 Date is not null & Coupon_id is null, 則為普通消費日期;
#        return 0
#    else:    ### 若 Date is not null & Coupon_id is not null, 則表示優惠券消費日期)
#        return 1

#dfoff["couponUsage"] = dfoff.apply(getCouponUsage, axis=1)
#dftest["couponUsage"] = dftest.apply(getCouponUsage, axis=1)
#dfoff["couponUsage"].value_counts()

In [10]:
# Generate features - 用戶數擁有的Coupon數
dfoff['userAmount'] = dfoff.groupby('User_id')['User_id'].transform('count')
dftest['userAmount'] = dftest.groupby('User_id')['User_id'].transform('count')

In [11]:
# Generate features - 店家發放的Coupon數
dfoff['merchantCouponAmount'] = dfoff.groupby('Merchant_id')['Merchant_id'].transform('count')
dftest['merchantCouponAmount'] = dftest.groupby('Merchant_id')['Merchant_id'].transform('count')

def getMoreThan100Coupon(row):
    if row > 100:
        return row / 100
    else:
        return 0
    
dfoff['merchantCouponAmount']  = dfoff['merchantCouponAmount'].apply(getMoreThan100Coupon)
dftest['merchantCouponAmount'] = dftest['merchantCouponAmount'].apply(getMoreThan100Coupon)

In [12]:
# Generate features - 消費時是否有持有Coupon
def getHaveCoupon(row):
    if np.isnan(row):
        return 0
    else:
        return 1
    
dfoff["haveCoupon"] = dfoff['Coupon_id'].apply(getHaveCoupon)
dftest["haveCoupon"] = dftest['Coupon_id'].apply(getHaveCoupon)

In [13]:
# Generate features - 持有店家Coupon的數量
dfoff['shopEnterUsers'] = dfoff.groupby('Merchant_id')['haveCoupon'].transform('count')
dftest['shopEnterUsers'] = dftest.groupby('Merchant_id')['haveCoupon'].transform('count')

In [14]:
# Generate features - coupon discount and distance
def getDiscountType(row):
    if row == 'null':
        return 'null'
    elif ':' in row:
        return 1
    else:
        return 0

def convertRate(row):
    """Convert discount to rate"""
    if row == 'null':
        return 1.0
    elif ':' in row:
        rows = row.split(':')
        return 1.0 - float(rows[1])/float(rows[0])
    else:
        return float(row)

def getDiscountMan(row):  #分母
    if ':' in row:
        rows = row.split(':')
        return int(rows[0])
    else:
        return 0

def getDiscountJian(row):  #分子
    if ':' in row:
        rows = row.split(':')
        return int(rows[1])
    else:
        return 0
    



def processData(df):
    
    # convert discunt_rate
    df['discount_rate'] = df['Discount_rate'].astype('str').apply(convertRate)
    df['discount_man'] = df['Discount_rate'].astype('str').apply(getDiscountMan)
    df['discount_jian'] = df['Discount_rate'].astype('str').apply(getDiscountJian)
    df['discount_type'] = df['Discount_rate'].astype('str').apply(getDiscountType)
    
    # convert distance
    df.loc[df.Distance.isna(), "Distance"] = 99
    df['Distance'] = df['Distance'] + 1 # 讓 distance 最小為1
    
    # Generate features - 越遠需要越高的折扣率
    df['discountFromDistance'] = df['discount_rate'] / df['Distance'] #distance不為零才可以除
    
    # Generate features - 找出持有Coupon時客戶與店家的距離
    df['merchantDistance'] = df['Distance'] * df['haveCoupon']
    
    # Generate features - 有意願跑到這家店的程度 (發放數量與距離關係)
    df['amountDistance'] = df['merchantCouponAmount'] / df['Distance']
    
    return df

dfoff = processData(dfoff)
dftest = processData(dftest)

In [15]:
dfoff.head(20)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label,weekday,weekday_type,...,merchantCouponAmount,haveCoupon,shopEnterUsers,discount_rate,discount_man,discount_jian,discount_type,discountFromDistance,merchantDistance,amountDistance
0,1439408,2632,NaN,NaN,1.0,NaN,20160217.0,-1,NaN,0,...,0.00,0,45,NaN,0,0,0,NaN,0.0,0.000000
1,1439408,2632,8591.0,20:1,1.0,20160217.0,NaN,0,3.0,0,...,0.00,1,45,0.950000,20,1,1,0.950000,1.0,0.000000
2,1439408,2632,1078.0,20:1,1.0,20160319.0,NaN,0,6.0,0,...,0.00,1,45,0.950000,20,1,1,0.950000,1.0,0.000000
3,1832624,3381,7610.0,200:20,1.0,20160429.0,NaN,0,5.0,0,...,1370.54,1,137054,0.900000,200,20,1,0.900000,1.0,1370.540000
4,2029232,3381,11951.0,200:20,2.0,20160129.0,NaN,0,5.0,0,...,1370.54,1,137054,0.900000,200,20,1,0.450000,2.0,685.270000
5,2223968,3381,9776.0,10:5,3.0,20160129.0,NaN,0,5.0,0,...,1370.54,1,137054,0.500000,10,5,1,0.166667,3.0,456.846667
6,73611,2099,12034.0,100:10,100.0,20160207.0,NaN,0,7.0,0,...,216.09,1,21609,0.900000,100,10,1,0.009000,100.0,2.160900
7,163606,1569,5054.0,200:30,11.0,20160421.0,NaN,0,4.0,0,...,341.42,1,34142,0.850000,200,30,1,0.077273,11.0,31.038182
8,3273056,4833,7802.0,200:20,11.0,20160130.0,NaN,0,6.0,0,...,87.91,1,8791,0.900000,200,20,1,0.081818,11.0,7.991818
9,94107,3381,7610.0,200:20,3.0,20160412.0,NaN,0,2.0,0,...,1370.54,1,137054,0.900000,200,20,1,0.300000,3.0,456.846667


In [16]:
## Naive model
def split_train_valid(row, date_cut="20160416"):
    is_train = True if pd.to_datetime(row, format="%Y%m%d") < pd.to_datetime(date_cut, format="%Y%m%d") else False
    return is_train
    
df = dfoff[dfoff['label'] != -1].copy()
df["is_train"] = df["Date_received"].apply(split_train_valid)
train = df[df["is_train"]]
valid = df[~df["is_train"]]
train.reset_index(drop=True, inplace=True)
valid.reset_index(drop=True, inplace=True)
print("Train size: {}, #positive: {}".format(len(train), train["label"].sum()))
print("Valid size: {}, #positive: {}".format(len(valid), valid["label"].sum()))

Train size: 667753, #positive: 32472
Valid size: 79216, #positive: 3832


In [17]:
df.columns

Index(['User_id', 'Merchant_id', 'Coupon_id', 'Discount_rate', 'Distance',
       'Date_received', 'Date', 'label', 'weekday', 'weekday_type',
       'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5',
       'weekday_6', 'weekday_7', 'usedOntime', 'userAmount',
       'merchantCouponAmount', 'haveCoupon', 'shopEnterUsers', 'discount_rate',
       'discount_man', 'discount_jian', 'discount_type',
       'discountFromDistance', 'merchantDistance', 'amountDistance',
       'is_train'],
      dtype='object')

In [18]:
df.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label,weekday,weekday_type,...,haveCoupon,shopEnterUsers,discount_rate,discount_man,discount_jian,discount_type,discountFromDistance,merchantDistance,amountDistance,is_train
1,1439408,2632,8591.0,20:1,1.0,20160217.0,NaN,0,3.0,0,...,1,45,0.95,20,1,1,0.950000,1.0,0.000000,True
2,1439408,2632,1078.0,20:1,1.0,20160319.0,NaN,0,6.0,0,...,1,45,0.95,20,1,1,0.950000,1.0,0.000000,True
3,1832624,3381,7610.0,200:20,1.0,20160429.0,NaN,0,5.0,0,...,1,137054,0.90,200,20,1,0.900000,1.0,1370.540000,False
4,2029232,3381,11951.0,200:20,2.0,20160129.0,NaN,0,5.0,0,...,1,137054,0.90,200,20,1,0.450000,2.0,685.270000,True
5,2223968,3381,9776.0,10:5,3.0,20160129.0,NaN,0,5.0,0,...,1,137054,0.50,10,5,1,0.166667,3.0,456.846667,True


In [19]:
original_feature = ['discount_rate',
                    'discount_type',
                    'discount_man', 
                    'discount_jian',
                    'Distance', 
                    'weekday', 
                    'discountFromDistance',
                    'usedOntime',
                    'merchantCouponAmount',
                    'merchantDistance',
                    'amountDistance',
                    'shopEnterUsers',
                    'userAmount',
                    'haveCoupon',
#                    'couponUsage',
                    'weekday_type'] + weekdaycols
print(len(original_feature),original_feature)

22 ['discount_rate', 'discount_type', 'discount_man', 'discount_jian', 'Distance', 'weekday', 'discountFromDistance', 'usedOntime', 'merchantCouponAmount', 'merchantDistance', 'amountDistance', 'shopEnterUsers', 'userAmount', 'haveCoupon', 'weekday_type', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']


In [20]:
predictors = original_feature
print(predictors)

def check_model(data, predictors):
    
    classifier = lambda: SGDClassifier(
        loss='log', 
        penalty='elasticnet', 
        fit_intercept=True, 
        max_iter=100, 
        shuffle=True, 
        n_jobs=1,
        class_weight=None)

    model = Pipeline(steps=[
        ('ss', StandardScaler()),
        ('en', classifier())
    ])

    parameters = {
        'en__alpha': [ 0.001, 0.01, 0.1],
        'en__l1_ratio': [ 0.001, 0.01, 0.1]
    }

    folder = StratifiedKFold(n_splits=5, shuffle=True)
    
    grid_search = GridSearchCV(
        model, 
        parameters, 
        cv=folder, 
        n_jobs=-1, 
        verbose=1)
    grid_search = grid_search.fit(data[predictors], 
                                  data['label'])
    
    return grid_search

['discount_rate', 'discount_type', 'discount_man', 'discount_jian', 'Distance', 'weekday', 'discountFromDistance', 'usedOntime', 'merchantCouponAmount', 'merchantDistance', 'amountDistance', 'shopEnterUsers', 'userAmount', 'haveCoupon', 'weekday_type', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']


In [21]:
model = check_model(train, predictors)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:  5.6min finished
/Users/dragonfly-mac/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/dragonfly-mac/anaconda3/lib/python3.7/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/Users/dragonfly-mac/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change i

In [22]:
y_valid_pred = model.predict_proba(valid[predictors])
valid1 = valid.copy()
valid1['pred_prob'] = y_valid_pred[:, 1]

/Users/dragonfly-mac/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:381: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [23]:
from sklearn.metrics import roc_auc_score, accuracy_score
auc_score = roc_auc_score(y_true=valid.label, y_score=y_valid_pred[:,1])
acc = accuracy_score(y_true=valid.label, y_pred=y_valid_pred.argmax(axis=1))
print("Validation AUC: {:.3f}, Accuracy: {:.3f}".format(auc_score, acc))

Validation AUC: 0.812, Accuracy: 0.952


compare upper cell and lower cell

## origin one
Validation AUC: 0.743, Accuracy: 0.952
## + usedOntime, getDiscountFromDistance
Validation AUC: 0.773, Accuracy: 0.952
## + merchantCouponAmount,  merchantDistance, amountDistance
Validation AUC: 0.785, Accuracy: 0.952
## + UserAmount
Validation AUC: 0.806, Accuracy: 0.952
## + shopEnterUsers, haveCoupon, new merchantCouponAmount
Validation AUC: 0.812, Accuracy: 0.952

In [24]:
targetset = dftest.copy()
print(targetset.shape)
targetset = targetset[~targetset.Coupon_id.isna()]
targetset.reset_index(drop=True, inplace=True)
testset = targetset[predictors].copy()

y_test_pred = model.predict_proba(testset[predictors])
test1 = testset.copy()
test1['pred_prob'] = y_test_pred[:, 1]
print(test1.shape)

(306313, 27)
(306313, 23)


/Users/dragonfly-mac/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:381: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [25]:
output = pd.concat((targetset[["User_id", "Coupon_id", "Date_received"]], test1["pred_prob"]), axis=1)
print(output.shape)

output.loc[:, "User_id"] = output["User_id"].apply(lambda x:str(int(x)))
output.loc[:, "Coupon_id"] = output["Coupon_id"].apply(lambda x:str(int(x)))
output.loc[:, "Date_received"] = output["Date_received"].apply(lambda x:str(int(x)))
output["uid"] = output[["User_id", "Coupon_id", "Date_received"]].apply(lambda x: '_'.join(x.values), axis=1)
output.reset_index(drop=True, inplace=True)

(306313, 4)


In [26]:
### NOTE: YOUR SUBMITION FILE SHOULD HAVE COLUMN NAME: uid, label
out = output.groupby("uid", as_index=False).mean()
out = out[["uid", "pred_prob"]]
out.columns = ["uid", "label"]
out.to_csv("baseline_example.csv", header=["uid", "label"], index=False) # submission format
out.head()

,uid,label
0,1000020_2705_20160519,0.072150
1,1000020_8192_20160513,0.073694
2,1000065_1455_20160527,0.065318
3,1000085_8067_20160513,0.067379
4,1000086_2418_20160613,0.061163


In [27]:
out.shape

(304096, 2)